<a href="https://colab.research.google.com/github/ad451/Stackoverflow_tag_generator/blob/main/StackOverflow_Tag_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Importing the required modules**

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import random
from bs4 import BeautifulSoup
import time
import warnings; warnings.simplefilter('ignore')
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
import gensim
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


nltk.download('averaged_perceptron_tagger') # required for parts of speech
nltk.download('wordnet') # required for parts of speech
nltk.download('stopwords') #download the stopwords


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Web Scraping the current set of questions for testing the model**

In [ ]:


Questions=[] #array to store the questions

for pageNumber in range(1,401):
    response=requests.get("https://stackoverflow.com/questions",params={"tab":"newest","page":pageNumber,"pagesize":50})

    data=BeautifulSoup(response.text,'html.parser' ) #parsing the html text

    req_data=data.find(id="questions")

    new_data=req_data.find_all("h3", class_="s-post-summary--content-title") # the tag that contains the question info



    for element in new_data:
        link=element.a.attrs['href']

        response=requests.get(f"https://stackoverflow.com/{link}")  #fetching the content related to the question

        data_questionwise=BeautifulSoup(response.text,'html.parser')

        question_wise_title=data_questionwise.find("div",id="question-header").h1.a.string #title

        question_wise_desc=data_questionwise.find("div",class_="s-prose js-post-body") #description

        all_paragraphs=question_wise_desc.find_all("p")

        total_description_question_wise=""

        for para in all_paragraphs:
            total_description_question_wise+=para.text

        Final_content=question_wise_title+""+total_description_question_wise  #concatenating the title and description

        tag_question_wise=data_questionwise.find("ul",class_="ml0 list-ls-none js-post-tag-list-wrapper d-inline").li.text #tag

        Questions.append([Final_content,tag_question_wise])

    print(pageNumber)   #checking which page questions have been fetched yet

print(len(Questions))








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.2 MB/s eta 0:00:00


**Sql code to combine the input questions and tags table**

In [ ]:
###################################### Code for inner combine ######################################
df1 = pd.read_csv('Questions.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('Tags.csv', encoding='ISO-8859-1')

# combined dataframe of questiontags
df3 = df1.set_index('Id').join(df2.set_index('Id'))
df3 = df3.reset_index()
df4 = pd.DataFrame(df3.Id.value_counts())
df4 = df4.reset_index()

# df5 is the final combined dataframe of questiontags. Randomly picked datapoints from df3
store = {}
df5 = pd.DataFrame()
n_train = 20000
for i in range(n_train):
    key = 0
    while(key==0):
        a = random.randint(df4.shape[0])
        if a in store:
            continue
        else:
            key==1
            store[a] = 1
            ind = df3[df3.Id==df4.iloc[a,0]].first_valid_index()
            df5 =  pd.concat([df5,pd.DataFrame(df3.iloc[ind]).T],ignore_index=True)
            break

###################################### Code for removing angular brackets from the "Body" column ######################################

for j in range(df5.shape[0]):
    ans=" "
    curr = 0
    flag = 0
    while(curr<len(df5['Body'][j])):
        if( df5['Body'][j][curr]=='<'):
            flag = 1
            curr = curr + 1
            continue
        if( df5['Body'][j][curr]=='>'):
            flag = 0
            curr = curr + 1
            continue
        if(flag == 1):
            curr = curr + 1
        if(flag == 0):
            ans = ans + df5['Body'][j][curr]
            curr = curr + 1
    df1['Title'][j] += ans

df6 = pd.concat([df5.iloc[:,6], df5.iloc[:,8]], axis=1)

**Machine learning part (Preprocessing and training)**

---



In [ ]:
train_data=pd.read_csv("questiontags_train.csv")
test_data=pd.read_csv("questiontags_test.csv")

In [ ]:
#rename the columns of the train dataset

train_data.drop(train_data.columns[0],axis=1,inplace=True)


train_data.rename(columns={"Title":"Questions","Tag":"Tags"},inplace=True)


In [ ]:
import random
#code to include around 10000 more questions to the train dataset from the test dataset
v={}
count=0
while count!=10000:
    idx=random.randint(0,test_data.shape[0]-1)
    if idx not in v:
      train_data=train_data.append(test_data.loc[idx],ignore_index=True)
      test_data.drop(idx,inplace=True,axis=0)
      count+=1
      v[idx]=1
    else:
      continue


In [ ]:
train_data['Questions']=train_data['Questions'].str.replace('</p>',' ')
train_data['Questions']=train_data['Questions'].str.replace('\n',' ')
train_data['Questions']=train_data['Questions'].str.replace('</a>',' ')
train_data['Questions']=train_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls
train_data['Questions'] = train_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3])) #removes small length words (len<3)
train_data['Questions']=train_data['Questions'].str.replace('<a href=" ">','')

train_data['Questions']=train_data['Questions'].apply(lambda x : x.lower()) #coverting to lowercase


In [ ]:
test_data['Questions']=test_data['Questions'].str.replace('</p>',' ')
test_data['Questions']=test_data['Questions'].str.replace('\n',' ')
test_data['Questions']=test_data['Questions'].str.replace('</a>',' ')
test_data['Questions'] = test_data['Questions'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
test_data['Questions']=test_data['Questions'].str.replace('<a href=" ">','')
test_data['Questions']=test_data['Questions'].apply(lambda x : x.lower())
test_data['Questions']=test_data['Questions'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)) #removing any urls

In [ ]:
#removing the stop words and the punctuations from test and train dataset


stop_words=set(nltk.corpus.stopwords.words('english'))
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([w for w in x.split() if w not in stop_words]))

In [ ]:
#removing the punctuations from test and train dataset


punctuations = string.punctuation
train_data['Questions']=train_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))
test_data['Questions']=test_data['Questions'].apply(lambda x:' '.join([''.join([char for char in w if char not in punctuations]) for w in x.split()]))



In [ ]:
test_data=test_data.reset_index()


In [ ]:

test_data.drop(['index'],inplace=True,axis=1)

In [ ]:
#dropping the rows with empty values of Question after filtering
for j in range(len(test_data['Questions'])):
  if len(test_data['Questions'][j])==0:
     test_data.drop(j,inplace=True)

for j in range(len(train_data['Questions'])):
  if len(train_data['Questions'][j])==0:
     train_data.drop(j,inplace=True)
     print("yes")

yes
yes
yes
yes
yes


In [ ]:
test_data=test_data.reset_index()
train_data=train_data.reset_index()

In [ ]:
test_data.drop(['index'],inplace=True,axis=1)
train_data.drop(['index'],inplace=True,axis=1)


In [ ]:
#using lemmatization on the questions of the train and test dataset
def lemmatization(text):
    pos_dict = {
        'N': 'n',  # Noun
        'V': 'v',  # Verb
        'R': 'r',  # Adverb
        'J': 'a'   # Adjective
    }
    pos_tags = pos_tag(text)
    lemmatizer = WordNetLemmatizer()
    lemma=[]
    for word, tag in pos_tags:
        if (tag[0].upper() not in pos_dict.keys()):
          pos='n'
        else:
          pos= pos_dict[tag[0].upper()]
        lemma.append(lemmatizer.lemmatize(word,pos=pos))
    return lemma

train_data['Questions']=train_data['Questions'].apply(lambda x : lemmatization(x.split()))
test_data['Questions']=test_data['Questions'].apply(lambda x : lemmatization(x.split()))
train_data["Questions"]=train_data["Questions"].apply(lambda x : " ".join(x))

test_data["Questions"]=test_data["Questions"].apply(lambda x : " ".join(x))


**Data Exploration with dev of Base Model**

In [ ]:
questions = train_data['Questions'].tolist()
tags = train_data['Tags'].tolist()

print('The total number of words in the data is: ', sum([len(text.split()) for text in questions]))

def tokenize_question(text):
    return text.split()

question_vect = CountVectorizer(tokenizer=tokenize_question)
questions=question_vect.fit_transform(questions)

print('The number of words in the vocabulary is: ', len(question_vect.vocabulary_))

def tokenize_tags(text):
    return text.split('|')

max_tags = 100
tags_vect = CountVectorizer(tokenizer=tokenize_tags, max_features=max_tags)
tags = tags_vect.fit_transform(tags)
tags = tags.toarray()
print('Number of tags: ', len(tags_vect.vocabulary_))

tags_token = tags_vect.get_feature_names_out()
tag_frequency = tags.sum(axis=0)
print('The list of tags with frequency is: ')
print(dict(zip(tags_token, tag_frequency)))



The total number of words in the data is:  2201637
The number of words in the vocabulary is:  269815
Number of tags:  100
The list of tags with frequency is: 
{'.htaccess': 30, '.net': 156, 'actionscript-3': 39, 'ajax': 42, 'algorithm': 68, 'amazon-web-services': 125, 'android': 1402, 'angular': 106, 'angularjs': 163, 'apache': 52, 'apache-spark': 40, 'arrays': 53, 'asp.net': 216, 'asp.net-mvc': 102, 'assembly': 27, 'azure': 109, 'bash': 77, 'batch-file': 32, 'c': 402, 'c#': 2015, 'c++': 991, 'cakephp': 28, 'css': 263, 'database': 84, 'delphi': 53, 'django': 145, 'docker': 109, 'eclipse': 65, 'elasticsearch': 35, 'excel': 208, 'facebook': 53, 'firebase': 34, 'flutter': 212, 'git': 152, 'github': 30, 'go': 67, 'google-apps-script': 40, 'hadoop': 42, 'haskell': 35, 'html': 488, 'html5': 39, 'ios': 674, 'iphone': 279, 'java': 2350, 'javascript': 2883, 'jquery': 530, 'jsf': 27, 'json': 90, 'kubernetes': 49, 'laravel': 57, 'linux': 201, 'machine-learning': 30, 'magento': 29, 'matlab': 87, '

In [ ]:
#checking the top most tags based on the frequency

tag_count = [[x[0],x[1]] for x in zip(tags_token, tag_frequency)]
tag_count = sorted(tag_count, key = lambda x: x[1], reverse=True)
most_common_tags=[tag[0] for tag in tag_count]
print('Rank     Tag      Count')
for i in range(100):
    print("%d   %s   %d" % (i+1, tag_count[i][0], tag_count[i][1]))

Rank     Tag      Count
1   javascript   2883
2   java   2350
3   python   2322
4   c#   2015
5   php   1705
6   android   1402
7   c++   991
8   ios   674
9   jquery   530
10   html   488
11   r   472
12   sql   469
13   c   402
14   ruby-on-rails   389
15   mysql   368
16   node.js   333
17   reactjs   297
18   iphone   279
19   css   263
20   asp.net   216
21   flutter   212
22   excel   208
23   linux   201
24   objective-c   172
25   angularjs   163
26   .net   156
27   git   152
28   django   145
29   ruby   143
30   windows   141
31   sql-server   137
32   amazon-web-services   125
33   swift   122
34   azure   109
35   docker   109
36   angular   106
37   python-3.x   104
38   typescript   103
39   asp.net-mvc   102
40   regex   99
41   wordpress   97
42   xml   96
43   vb.net   95
44   json   90
45   postgresql   90
46   matlab   87
47   database   84
48   spring   83
49   wpf   83
50   scala   81
51   powershell   78
52   bash   77
53   mongodb   74
54   algorithm   68
55   g

In [ ]:
# X_test = test_data['Questions'].tolist()
# y_test = test_data['Tags'].tolist()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(questions, tags,
#                                                     test_size=0.2,
#                                                     random_state=747)

# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
#                                                     test_size=0.5,
#                                                     random_state=747)

In [ ]:
#function to calculate the evaluation metrics

def eval_metrics(y_test, y_predicted, print_metrics=True):

    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')

    if print_metrics:
        print("f1: %.3f - precision: %.3f - recall: %.3f - accuracy: %.3f" % (
            f1, precision, recall, accuracy))
    return f1, precision, recall, accuracy

In [97]:
train_data.head()

train_new=train_data.copy()
test_new=test_data.copy()

In [98]:


def tokenize_question(text):
    return text.split()

def filter_number_features(name):
  if name[0] in '0123456789' or len(name)<=3:
    return False
  return True
tfidf_vect = TfidfVectorizer(tokenizer=tokenize_question,
                               stop_words='english',
                               min_df=4,
                               max_df=0.5,max_features=7000)

X_train_tfidf = tfidf_vect.fit_transform(train_new["Questions"]).todense()
X_test_tfidf = tfidf_vect.transform(test_new["Questions"]).todense()
# print('The number of words in the vocabulary is: ', len(tfidf_vect.vocabulary_))



#get the feature names
feature_names=tfidf_vect.get_feature_names_out()

# Get the IDF scores
idf_scores = tfidf_vect.idf_

Final_Feature_Set=[]
for idx,feature_name in enumerate(feature_names):
    if filter_number_features(feature_name):
       Final_Feature_Set.append([feature_name,idf_scores[idx]])

Final_Feature_Set=sorted(Final_Feature_Set,key=lambda x :x[1],reverse=True)
Final_Feature_Set=[x[0] for x in Final_Feature_Set]


In [99]:
def tokenize_question(text):
      return text.split()

def CountVectorizer_Custom(data,Features):

  questions = data['Questions'].tolist()
  tags = data['Tags'].tolist()

  question_vect = CountVectorizer(tokenizer=tokenize_question,binary=True,vocabulary=Features)
  questions=question_vect.fit_transform(questions)

  df_train = pd.DataFrame(questions.toarray(), columns=question_vect.get_feature_names_out())
  df_train["Tags"]=tags
  return df_train

train_new=CountVectorizer_Custom(train_new,Final_Feature_Set)
test_new=CountVectorizer_Custom(test_new,Final_Feature_Set)









In [ ]:
def filter_data_by_most_common_tags(data, common_tags):
    filtered_data = data[data["Tags"].isin(common_tags)]
    return filtered_data


def one_hot(column,data):
  # Perform one-hot encoding using get_dummies()
  one_hot_encoded = pd.get_dummies(data[column],prefix="tag")

  # Concatenate the one-hot encoded columns with the original dataframe
  data_extended = pd.concat([data, one_hot_encoded], axis=1)
  data_extended.drop(['Tags'],inplace=True,axis=1)

  return data_extended


Final_train=filter_data_by_most_common_tags(train_new,most_common_tags)
Final_test=filter_data_by_most_common_tags(test_new,most_common_tags)

#dealing with missing tags

missing_tags = set(Final_train["Tags"]) - set(Final_test["Tags"])

Final_train=one_hot("Tags",Final_train)

Final_test=one_hot("Tags",Final_test)

for tag in missing_tags:
    Final_test["tag_"+tag] = 0

Final_test=Final_test[Final_train.columns] # in order to maintain order of the columns in test and train


In [ ]:
X_train=Final_train.iloc[:,:-1*len(most_common_tags)]

Y_train=Final_train.iloc[:,-1*len(most_common_tags):]
X_test=Final_test.iloc[:,:-1*len(most_common_tags)]

Y_test=Final_test.iloc[:,-1*len(most_common_tags):]

In [ ]:
%%time
tfidf_log_clf = OneVsRestClassifier(LogisticRegression())

tfidf_log_clf.fit(X_train_tfidf, y_train)

CPU times: user 27.6 s, sys: 28.5 s, total: 56.1 s
Wall time: 32 s


OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
y_train_tfidf_predict = tfidf_log_clf.predict(X_train_tfidf)
eval_metrics(y_train, y_train_tfidf_predict)

f1: 0.287 - precision: 0.705 - recall: 0.192 - accuracy: 0.325


(0.2869979496204687,
 0.7048346764490622,
 0.19239736441966548,
 0.3247759949989581)

In [ ]:
y_test_tfidf_predict = tfidf_log_clf.predict(X_test_tfidf)
eval_metrics(y_test, y_test_tfidf_predict)

f1: 0.246 - precision: 0.563 - recall: 0.164 - accuracy: 0.287


(0.24568413711051668, 0.5629343082208758, 0.1643342662934826, 0.287)

In [ ]:
tfidf_log_clf = OneVsRestClassifier(LogisticRegression())

tfidf_log_clf.fit(X_train, Y_train)



TypeError: ignored

In [ ]:
Y_train_predict = tfidf_log_clf.predict(X_train)
eval_metrics(Y_train, Y_train_predict)

f1: 0.817 - precision: 0.953 - recall: 0.719 - accuracy: 0.697


(0.816908226438034, 0.9532802106957978, 0.7186436558094312, 0.6970507211148922)

In [ ]:
Y_test_predict=tfidf_log_clf.predict(X_test)
eval_metrics(Y_test,Y_test_predict)

ValueError: ignored

In [95]:
column1=X_train.columns
column2=X_test.columns

print(len(column1))
print(len(column2))

6137
6137
